In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context

context = Context()

Start

In [ ]:
from contracts.pair_contract import PairContract

router = context.get_contracts(config.ROUTER_V2)[0]
pair_contract: PairContract
contracts = context.get_contracts(config.PAIRS_V2)
context.deployer_account.sync_nonce(context.network_provider.proxy)
for pair_contract in contracts:
    router.pair_contract_resume(context.deployer_account, context.network_provider.proxy, pair_contract.address)

In [ ]:
from contracts.staking_contract import StakingContract
staking_contract: StakingContract
staking_contract = context.get_contracts(config.STAKINGS_V2)[0]

In [ ]:
from contracts.metastaking_contract import MetaStakingContract
metastaking_contract: MetaStakingContract
metastaking_contract = context.get_contracts(config.METASTAKINGS_V2)[4]

Energy contract setup

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

In [ ]:
energy_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.SIMPLE_LOCK_ENERGY_BYTECODE_PATH, [], True)

In [ ]:
energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, 
                                    metastaking_contract.address)

In [ ]:
energy_contract.resume(context.deployer_account, context.network_provider.proxy)

In [ ]:
energy_contract.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, ["erd1qqqqqqqqqqqqqpgq0lmttcgmxjx5arcvtgmvsle23qhch6052jps0skg6f"])

In [ ]:
energy_contract.add_sc_to_token_transfer_whitelist(context.deployer_account, context.network_provider.proxy, ["erd1qqqqqqqqqqqqqpgq0lmttcgmxjx5arcvtgmvsle23qhch6052jps0skg6f"])

In [ ]:
energy_contract.pause(context.deployer_account, context.network_provider.proxy)

Set energy for user

In [ ]:
energy_contract.set_energy_for_old_tokens(context.deployer_account, context.network_provider.proxy, 
                                          ["erd1avdrs0nkqapufu7zpmufv6kqtqnr6euwsuf7e57cu0cmq79p3ryqtyxqxj", 4726684797026807555315772, -836623209073744937290891644])

In [ ]:
from utils.utils_chain import Account, WrapperAddress
from utils.utils_tx import ESDTToken
user = Account(pem_file=config.DEFAULT_ADMIN)
user.address = WrapperAddress("erd1avdrs0nkqapufu7zpmufv6kqtqnr6euwsuf7e57cu0cmq79p3ryqtyxqxj")
user.sync_nonce(context.network_provider.proxy)
xmex = ESDTToken("XMEX-fda355", 28, 4726684797026807555315772)
energy_contract.unlock_tokens(user, context.network_provider.proxy, [[xmex]])

In [ ]:
txhash = staking_contract.register_farm_token(context.deployer_account, context.network_provider.proxy, ["STKTST", "STKTST"])
status = context.network_provider.check_simple_tx_status(txhash, "set boosted yields factors in farm")

In [ ]:
status = context.network_provider.check_simple_tx_status("5f5e3b111bc5ca2f59e4f8bb8faf3aa7e7827bc0247808b53eb7bd58e9db19e4", "set boosted yields factors in farm")

In [ ]:
from utils.contract_data_fetchers import MetaStakingContractDataFetcher
from utils.utils_chain import WrapperAddress as Address, hex_to_string
farm_token_hex = MetaStakingContractDataFetcher(Address(metastaking_contract.address), context.network_provider.proxy.url).get_data("getDualYieldTokenId")
print(hex_to_string(farm_token_hex))

In [ ]:
from contracts.pair_contract import PairContract
pair_contract: PairContract
pair_contracts = context.get_contracts(config.PAIRS_V2)
for pair_contract in pair_contracts:
    pair_contract.resume(context.deployer_account, context.network_provider.proxy)

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
router_contract.set_fee_on(context.deployer_account, context.network_provider.proxy, 
                           [
                                "erd1qqqqqqqqqqqqqpgqkdxw8ewquxtt64t8rmdgqz3ja0zjycfm295qpdm8at",
                                "erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu",
                                "MEX-4638e4"
                           ])

In [ ]:
from contracts.router_contract import RouterContract
from tools.account_state import get_account_keys_online, report_key_files_compare

router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]

get_account_keys_online(router_contract.address, context.network_provider.proxy.url, with_save_in="state_dump/pre_upgrade_router.json")
router_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy,
                                 config.ROUTER_V2_BYTECODE_PATH,
                                 ["erd1qqqqqqqqqqqqqpgqj7yq0qysvn2xye0ywzkl0qn95wz8ttcr2jpscpfazv"])

In [ ]:
get_account_keys_online(router_contract.address, context.network_provider.proxy.url, with_save_in="state_dump/post_upgrade_router.json")
report_key_files_compare("state_dump", "pre", "post", True)

FEES setup for pairs

In [ ]:
import time
from contracts.pair_contract import PairContract
from contracts.router_contract import RouterContract

router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]

pair_contract: PairContract
mex_contract = context.get_pair_contract_by_address("erd1qqqqqqqqqqqqqpgqsa0hwt0m05308mxlagc6wm3qs6duuh5f295qs5du2a") # egldmex contract

for sending_contract in context.get_contracts(config.PAIRS_V2):
    if sending_contract.address in ["erd1qqqqqqqqqqqqqpgqsa0hwt0m05308mxlagc6wm3qs6duuh5f295qs5du2a",
                                    "erd1qqqqqqqqqqqqqpgqkdxw8ewquxtt64t8rmdgqz3ja0zjycfm295qpdm8at"]:
        continue
    # set where to swap and what to do with the fees
    sending_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                        [
                                            mex_contract.address,
                                            mex_contract.firstToken,
                                            mex_contract.secondToken
                                        ])
    time.sleep(2)
    sending_contract.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                                    [
                                        "erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu",
                                            "MEX-4638e4"
                                    ])
    time.sleep(2)
    # whitelist swap no fees
    mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, sending_contract.address)
    time.sleep(2)

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
mex_contract.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                                   [
                                       "erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu",
                                        "MEX-4638e4"
                                   ])

In [ ]:
from contracts.fees_collector_contract import FeesCollectorContract

fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.FEES_COLLECTOR_BYTECODE_PATH, 
                                         [], 
                                         no_init=True)

addLiquidity

In [ ]:
from contracts.pair_contract import PairContract, AddLiquidityEvent, SwapFixedInputEvent
from utils.utils_chain import Account, WrapperAddress

pair_contract: PairContract
pair_contract = context.get_contracts(config.PAIRS_V2)[0]

user = Account(pem_file=config.DEFAULT_ADMIN)
user.address = WrapperAddress("erd195v2qrm9vvrr29t2ddp7zvyd4ylze8ujknxpzng0t2tcj4lexu9sdvjxt2")
user.sync_nonce(context.network_provider.proxy)

swap = SwapFixedInputEvent(pair_contract.firstToken, 10000000000000000, pair_contract.secondToken, 1)
pair_contract.swap_fixed_input(context.network_provider, user, swap)

In [ ]:
from utils.utils_tx import endpoint_call

endpoint_call(context.network_provider.proxy, 400000000, user, WrapperAddress(pair_contract.address), "MigrateDataTrie", [])

In [ ]:
from utils.utils_tx import endpoint_call
from time import sleep
from multiversx_sdk import ProxyNetworkProvider

# observer = ProxyNetworkProvider("https://165.232.72.116")
user = Account(pem_file=config.DEFAULT_ADMIN)
# user.address = WrapperAddress("erd1rf4hv70arudgzus0ymnnsnc4pml0jkywg2xjvzslg0mz4nn2tg7q7k0t6p")
user.address = WrapperAddress("erd1v4ms58e22zjcp08suzqgm9ajmumwxcy4hfkdc23gvynnegjdflmsj6gmaq")
user.sync_nonce(context.network_provider.proxy)

migated_query = "address/erd1lllllllllllllllllllllllllllllllllllllllllllllllllllsckry7t/is-data-trie-migrated"
response = context.network_provider.proxy.do_get_generic(migated_query)
while response.get("isMigrated") == False:
    for i in range(0, 50):
        endpoint_call(context.network_provider.proxy, 200000000, user, WrapperAddress("erd1lllllllllllllllllllllllllllllllllllllllllllllllllllsckry7t"), "MigrateDataTrie", [])
        sleep(0.5)

    target_nonce = user.nonce
    user.sync_nonce(context.network_provider.proxy)
    print(f"Waiting to reach nonce: {target_nonce}...")
    while user.nonce < target_nonce:
        sleep(6)
        user.sync_nonce(context.network_provider.proxy)
    
    response = context.network_provider.proxy.do_get_generic(migated_query)
    print(f"Current migration status: {response.get('isMigrated')}")